# 1. GEREKLİLİKLERİN YÜKLENMESİ VE AKTARILMASI

In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
!git clone https://github.com/ultralytics/yolov5 

In [ ]:
!cd yolov5 & pip install -r requirements.txt  

In [ ]:
import torch                                    #KÜTÜPHANELERİN İMPORT EDİLMESİ  
from matplotlib import pyplot as plt
import numpy as np
import cv2

# 2. YOLO MODELİNİN YÜKLENMESİ

In [ ]:
pip list

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')    #YOLOV5 HAZIR MODELİNİN YÜKLENMESİ 

In [ ]:
model

# 3. RESİMLERDEN TESPİT YAPMA

In [ ]:
img = 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Cars_in_traffic_in_Auckland%2C_New_Zealand_-_copyright-free_photo_released_to_public_domain.jpg/800px-Cars_in_traffic_in_Auckland%2C_New_Zealand_-_copyright-free_photo_released_to_public_domain.jpg'
# İNTERNETTEN ALINAN RESMİN URL'SİNİN 'img' ADLI DEĞİŞKENE ATANMASI

In [ ]:
results = model(img)    #MODELE RESMİN URLSİNİN VERİLMESİ VE SONUCUN YAZDIRILMASI 
results.print()

In [ ]:
%matplotlib inline 
plt.imshow(np.squeeze(results.render()))
plt.show()

In [ ]:
results.render()

# 4. GERÇEK ZAMANLI TESPİT YAPMA

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # TESPİT YAPILMASI  
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# 5. -----------------------CUSTOM MODEL OLUŞTURMA------------------------------------

In [ ]:
import uuid   # Benzersiz tanımlayıcı
import os
import time

In [ ]:
IMAGES_PATH = os.path.join('data', 'images') 
labels = ['UYANIK', 'YORGUN']
number_imgs = 20

# 6. DAHİLİ KAMERA İLE RESİMLERİN TOPLANMASI 

In [ ]:
cap = cv2.VideoCapture(0)
# Etiketler arasında geçiş yap
for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    
    # Görüntü aralığında döngü
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        
        # Webcam
        ret, frame = cap.read()
        
        # Görüntü yolunu adlandırma
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        
        # Görüntüyü dosyaya yazar 
        cv2.imwrite(imgname, frame)
        
        # Ekrana işleme
        cv2.imshow('Image Collection', frame)
        
        # Yakalamalar arasında 2 saniye gecikme
        time.sleep(2)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(os.path.join(IMAGES_PATH, labels[0]+'.'+str(uuid.uuid1())+'.jpg'))

In [ ]:
for label in labels:
    print('Collecting images for {}'.format(label))
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        print(imgname)   

# 7. 'labelImg' ADLI YAZILIMIN YÜKLENMESİ VE RESİMLERİN ETİKETLENMESİ

In [ ]:
!git clone https://github.com/tzutalin/labelImg  #GİTHUB'DAN KLONLAMA

In [ ]:
!pip install pyqt5 lxml --upgrade
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

# 8. ÖZEL MODELİN ETİKETLENMİŞ RESİMLER İLE EĞİTİLMESİ

In [ ]:
!cd yolov5 && python train.py --img 320 --batch 16 --epochs 100 --data dataset.yml --weights yolov5s.pt --workers 4
#EĞİTİMİN BAŞLATILMASI 

# 9. ÖZEL MODELİN YÜKLENMESİ

In [ ]:
import torch                                    #KÜTÜPHANELERİN İMPORT EDİLMESİ  
from matplotlib import pyplot as plt
import numpy as np
import cv2
import uuid   # Benzersiz tanımlayıcı
import os
import time

In [ ]:
pip install pygame

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp3/weights/last.pt', force_reload=True)
#SON GÜNCEL AĞIRLIKLARI KULLANARAK ÖZEL MODELİN YÜKLENMESİ 

# 10. ÖZEL MODEL İLE RESİMDEN TESPİT YAPMA

In [ ]:
img = os.path.join('data', 'images', 'UYANIK.7488fcc6-806b-11ed-bec3-b06ebf11babd.jpg') #BİLGİSAYARDAKİ RESİMDEN TEST EDİLMESİ

In [ ]:
results = model(img)

In [ ]:
results.print()

In [ ]:
%matplotlib inline 
plt.imshow(np.squeeze(results.render()))
plt.show()

# 11. ÖZEL MODEL İLE GERÇEK ZAMANLI TESPİT YAPILMASI

In [ ]:
import cv2
import numpy as np
import pygame
import time
import threading
import torch

# Assuming you have a YOLO model loaded as 'model'
# You may need to import the YOLO model and its dependencies

def play_alarm_sound():
    pygame.mixer.init()
    sound = pygame.mixer.Sound('alarm.wav')
    sound.play()
    while pygame.mixer.get_busy():
        pygame.time.Clock().tick(1)
    pygame.mixer.quit()

def alarm_thread():
    global alarm_baslangic_zamani
    alarm_baslangic_zamani = time.time()
    while True:
        play_alarm_sound()
        print("Alarm çalıyor... Dikkat! Yorgunsunuz!")
        time.sleep(1)
        if not cap.isOpened() or uyanik_durumu_algildi:
            break

# 2. Video Yakalama Başlatın
cap = cv2.VideoCapture(0)

# 3. Diğer Parametreleri Ayarlayın
skip_frames = 1  # Her 5 kareyi atla (gerektiğinde değiştirin)
count = 0

# Yorgunluk durumunu izlemek için bir değişken ata
yorgunluk_frame_sayisi = 0
yorgunluk_esik_degeri = 20  # Belirli bir sayıda ardışık yorgunluk durumu algılandığında alarmı tetikle

# Alarm durumunu kontrol etmek için bir değişken ata
alarm_aktif = False

# Alarm başlangıç zamanını takip etmek için bir değişken ata
alarm_baslangic_zamani = 0

# 4. Ana Döngü
while cap.isOpened():
    ret, frame = cap.read()

    # Her skip_frames karesini atla
    if count % skip_frames == 0:
        # Varsayılan olarak 'model' olarak yüklenmiş bir YOLO modeliniz olduğunu varsayalım
        results = model(frame)
        names = model.names

        # Nesne Tespitini Gerçekleştirin
        with torch.no_grad():  # Değerlendirme modunda çalıştır
            detections = results.xyxy[0]

            yorgun_durumu_algildi = False  # Yorgun durumu algılandı mı?
            uyanik_durumu_algildi = False  # Uyanık durumu algılandı mı?

            for detection in detections:
                label = names[int(detection[-1])]

                if label == 'YORGUN':
                    yorgunluk_frame_sayisi += 1

                    if yorgunluk_frame_sayisi >= yorgunluk_esik_degeri and not alarm_aktif:
                        # Alarm fonksiyonunu ayrı bir thread üzerinde çalıştır
                        alarm_thread_instance = threading.Thread(target=alarm_thread)
                        alarm_thread_instance.start()

                        # Yorgun durumu algılandı
                        yorgun_durumu_algildi = True

                        # Alarm aktif durumunu güncelle
                        alarm_aktif = True

                elif label == 'UYANIK':
                    uyanik_durumu_algildi = True

            # Eğer uyanık durumu algılandıysa yorgunluk frame sayısını sıfırla
            if uyanik_durumu_algildi:
                yorgunluk_frame_sayisi = 0
                alarm_aktif = False  # Uyanık durumu algılandığında veya eşik değeri aşıldığında alarmı kapat

            # Eğer alarm aktifse ve belirli bir süre geçtiyse "Dikkat! Yorgunsunuz!" yazısını göster
            if yorgunluk_frame_sayisi >= yorgunluk_esik_degeri:
                cv2.putText(frame, "Dikkat! Yorgunsunuz!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Her durumda frame sayısını göster
            message = f"Yorgunluk orani: {yorgunluk_frame_sayisi}"
            if yorgunluk_frame_sayisi > yorgunluk_esik_degeri:
                message = f"Yorgunluk orani: {yorgunluk_esik_degeri}"

            cv2.putText(frame, message, (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            if yorgunluk_frame_sayisi < yorgunluk_esik_degeri:
                # Çerçeve kalınlığını belirle, yorgunluk arttıkça kalınlığı artır
                frame_thickness = int(1 + 10 * np.exp(0.1 * yorgunluk_frame_sayisi))
                # Çerçeve rengi: (B, G, R), burada kırmızı renk
                frame_color = (0, 0, 255)
                # Çerçeve ekle
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), frame_color, frame_thickness)
            if yorgunluk_frame_sayisi > yorgunluk_esik_degeri:
                frame_thickness = int(1 + 10 * np.exp(0.1 * yorgunluk_esik_degeri))
                # Çerçeve rengi: (B, G, R), burada kırmızı renk
                frame_color = (0, 0, 255)
                # Çerçeve ekle
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), frame_color, frame_thickness)

    # 7. Sonuçları Göster
    cv2.imshow('YOLO', np.squeeze(results.render()))

    count += 1

    # 8. Çıkış İşlemi
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# 9. Temizleme
cap.release()
cv2.destroyAllWindows()

#deneme 4 

In [61]:
#KODUN SON HALI
import cv2
import numpy as np
import pygame
import time
import threading
import torch

# YOLO modelinin 'model' olarak yüklendiği varsayılıyor
# Gerekli YOLO model importları ve bağımlılıklarının düzgün şekilde yüklendiğinden emin olun

def play_alarm_sound():
    pygame.mixer.init()
    sound = pygame.mixer.Sound('alarm.wav')
    sound.play()
    while pygame.mixer.get_busy():
        pygame.time.Clock().tick(1)

def alarm_thread():
    global alarm_aktif
    alarm_aktif = True
    play_alarm_sound()
    print("Alarm çalıyor... Dikkat! Yorgunsunuz!")
    alarm_aktif = False

# Video yakalama başlatma
cap = cv2.VideoCapture(0)

# Kare atlama ve yorgunluk parametreleri
skip_frames = 3  # Performansı artırmak için gerektiği şekilde ayarlanabilir
yorgunluk_frame_sayisi = 0
yorgunluk_esik_degeri = 20

# Alarm durumu ve bayrakları
alarm_aktif = False

# Ana işleme döngüsü
count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Her 'skip_frames' kadar kare işleme
    if count % skip_frames == 0:
        with torch.no_grad():
            results = model(frame)
            names = model.names
            detections = results.xyxy[0]

            yorgun_durumu_algildi = False
            uyanik_durumu_algildi = False

            for detection in detections:
                label = names[int(detection[-1])]
                if label == 'YORGUN':
                    yorgunluk_frame_sayisi += 1
                    if yorgunluk_frame_sayisi >= yorgunluk_esik_degeri and not alarm_aktif:
                        alarm_thread_instance = threading.Thread(target=alarm_thread)
                        alarm_thread_instance.start()
                elif label == 'UYANIK':
                    uyanik_durumu_algildi = True

            # "Uyanık" durumu tespit edilirse yorgunluk sayacını sıfırlama
            if uyanik_durumu_algildi:
                yorgunluk_frame_sayisi = 0

            # Yorgunluk eşiği aşılırsa uyarı mesajı gösterme
            message = f"Yorgunluk orani: {yorgunluk_frame_sayisi}"
            if yorgunluk_frame_sayisi > yorgunluk_esik_degeri:
                message = f"Yorgunluk orani: {yorgunluk_esik_degeri}"
            # Eğer alarm aktifse ve belirli bir süre geçtiyse "Dikkat! Yorgunsunuz!" yazısını göster
            if yorgunluk_frame_sayisi >= yorgunluk_esik_degeri:
                cv2.putText(frame, "Dikkat! Yorgunsunuz!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.putText(frame, message, (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            # Yorgunluk seviyesine göre çerçeve kalınlığını ayarlama
            if yorgunluk_frame_sayisi < yorgunluk_esik_degeri:
                # Yorgunluk arttıkça çerçeve kalınlığını artırma
                frame_thickness = int(1 + 10 * np.exp(0.1 * yorgunluk_frame_sayisi))
                frame_color = (0, 0, 255)  # Kırmızı renk çerçeve için
                # Görüntü etrafına çerçeve ekleme
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), frame_color, frame_thickness)
            else:
                # Yorgunluk eşiğini geçtiğinde çerçeve kalınlığını sabitleme
                frame_thickness = int(1 + 10 * np.exp(0.1 * yorgunluk_esik_degeri))
                frame_color = (0, 0, 255)  # Kırmızı renk çerçeve için
                # Görüntü etrafına çerçeve ekleme
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), frame_color, frame_thickness)

    # Sonucu gösterme
    cv2.imshow('YOLO', np.squeeze(results.render()))

    count += 1

    # 'q' tuşuna basıldığında çıkma
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Temizlik işlemleri
cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()

#KODUN SON HALI

C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\comm

Alarm çalıyor... Dikkat! Yorgunsunuz!


C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\comm

Alarm çalıyor... Dikkat! Yorgunsunuz!


C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\comm

Alarm çalıyor... Dikkat! Yorgunsunuz!


C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Asus/.cache\torch\hub\ultralytics_yolov5_master\models\comm

In [ ]:
import cv2
import numpy as np
import pygame
import time
import threading
import torch

# Assuming you have a YOLO model loaded as 'model'
# You may need to import the YOLO model and its dependencies

def play_alarm_sound():
    pygame.mixer.init()
    sound = pygame.mixer.Sound('alarm.wav')
    sound.play()
    while pygame.mixer.get_busy():
        pygame.time.Clock().tick(1)
    pygame.mixer.quit()

def alarm_thread():
    global alarm_baslangic_zamani
    alarm_baslangic_zamani = time.time()
    while True:
        play_alarm_sound()
        print("Alarm çalıyor... Dikkat! Yorgunsunuz!")
        time.sleep(1)
        if not cap.isOpened() or uyanik_durumu_algildi:
            break

# 2. Video Yakalama Başlatın
cap = cv2.VideoCapture(0)

# 3. Diğer Parametreleri Ayarlayın
skip_frames = 1  # Her 5 kareyi atla (gerektiğinde değiştirin)
count = 0

# Yorgunluk durumunu izlemek için bir değişken ata
yorgunluk_frame_sayisi = 0
yorgunluk_esik_degeri = 20  # Belirli bir sayıda ardışık yorgunluk durumu algılandığında alarmı tetikle

# Alarm durumunu kontrol etmek için bir değişken ata
alarm_aktif = False

# Alarm başlangıç zamanını takip etmek için bir değişken ata
alarm_baslangic_zamani = 0

# 4. Ana Döngü
while cap.isOpened():
    ret, frame = cap.read()

    # Her skip_frames karesini atla
    if count % skip_frames == 0:
        # Varsayılan olarak 'model' olarak yüklenmiş bir YOLO modeliniz olduğunu varsayalım
        results = model(frame)
        names = model.names

        # Nesne Tespitini Gerçekleştirin
        detections = results.xyxy[0]

        yorgun_durumu_algildi = False  # Yorgun durumu algılandı mı?
        uyanik_durumu_algildi = False  # Uyanık durumu algılandı mı?

        for detection in detections:
            label = names[int(detection[-1])]

            if label == 'YORGUN':
                yorgunluk_frame_sayisi += 1

                if yorgunluk_frame_sayisi >= yorgunluk_esik_degeri and not alarm_aktif:
                    # Alarm fonksiyonunu ayrı bir thread üzerinde çalıştır
                    alarm_thread_instance = threading.Thread(target=alarm_thread)
                    alarm_thread_instance.start()

                    # Yorgun durumu algılandı
                    yorgun_durumu_algildi = True

                    # Alarm aktif durumunu güncelle
                    alarm_aktif = True

            elif label == 'UYANIK':
                uyanik_durumu_algildi = True

        # Eğer uyanık durumu algılandıysa yorgunluk frame sayısını sıfırla
        if uyanik_durumu_algildi:
            yorgunluk_frame_sayisi = 0
            alarm_aktif = False  # Uyanık durumu algılandığında veya eşik değeri aşıldığında alarmı kapat

        # Eğer alarm aktifse ve belirli bir süre geçtiyse "Dikkat! Yorgunsunuz!" yazısını göster
        if yorgunluk_frame_sayisi >= yorgunluk_esik_degeri:
            cv2.putText(frame, "Dikkat! Yorgunsunuz!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Her durumda frame sayısını göster
        message = f"Yorgunluk orani: {yorgunluk_frame_sayisi}"
        if yorgunluk_frame_sayisi > yorgunluk_esik_degeri:
            message = f"Yorgunluk orani: {yorgunluk_esik_degeri}"

        cv2.putText(frame, message, (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        if yorgunluk_frame_sayisi < yorgunluk_esik_degeri:
            # Çerçeve kalınlığını belirle, yorgunluk arttıkça kalınlığı artır
            frame_thickness = int(1 + 10 * np.exp(0.1 * yorgunluk_frame_sayisi))
            # Çerçeve rengi: (B, G, R), burada kırmızı renk
            frame_color = (0, 0, 255)
            # Çerçeve ekle
            cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), frame_color, frame_thickness)
        if yorgunluk_frame_sayisi > yorgunluk_esik_degeri:
            frame_thickness = int(1 + 10 * np.exp(0.1 * yorgunluk_esik_degeri))
            # Çerçeve rengi: (B, G, R), burada kırmızı renk
            frame_color = (0, 0, 255)
            # Çerçeve ekle
            cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), frame_color, frame_thickness)

    # 7. Sonuçları Göster
    cv2.imshow('YOLO', np.squeeze(results.render()))

    count += 1

    # 8. Çıkış İşlemi
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# 9. Temizleme
cap.release()
cv2.destroyAllWindows()

#deneme5  threshold yok ama skip frame dogru calısıyor 

In [ ]:
import cv2
import numpy as np
import pygame
import time
import threading
import torch

# Assuming you have a YOLO model loaded as 'model'
# You may need to import the YOLO model and its dependencies

def play_alarm_sound():
    pygame.mixer.init()
    sound = pygame.mixer.Sound('alarm.wav')
    sound.play()
    while pygame.mixer.get_busy():
        pygame.time.Clock().tick(1)
    pygame.mixer.quit()

def alarm_thread():
    global alarm_baslangic_zamani
    alarm_baslangic_zamani = time.time()
    while True:
        play_alarm_sound()
        print("Alarm çalıyor... Dikkat! Yorgunsunuz!")
        time.sleep(1)
        if not cap.isOpened() or uyanik_durumu_algildi:
            break

# 2. Video Yakalama Başlatın
cap = cv2.VideoCapture(0)

# 3. Diğer Parametreleri Ayarlayın
skip_frames = 1  # Her 5 kareyi atla (gerektiğinde değiştirin)
count = 0

# Yorgunluk durumunu izlemek için bir değişken ata
yorgunluk_frame_sayisi = 0
yorgunluk_esik_degeri = 20  # Belirli bir sayıda ardışık yorgunluk durumu algılandığında alarmı tetikle

# Alarm durumunu kontrol etmek için bir değişken ata
alarm_aktif = False

# Alarm başlangıç zamanını takip etmek için bir değişken ata
alarm_baslangic_zamani = 0

# Nesne tespiti için doğruluk eşiği (threshold)
confidence_threshold = 0.7

# 4. Ana Döngü
while cap.isOpened():
    ret, frame = cap.read()

    # Her skip_frames karesini atla
    if count % skip_frames == 0:
        # Varsayılan olarak 'model' olarak yüklenmiş bir YOLO modeliniz olduğunu varsayalım
        results = model(frame)
        names = model.names

        # Nesne Tespitini Gerçekleştirin
        detections = results.xyxy[0]

        yorgun_durumu_algildi = False  # Yorgun durumu algılandı mı?
        uyanik_durumu_algildi = False  # Uyanık durumu algılandı mı?

        for detection in detections:
            label = names[int(detection[-1])]
            confidence = float(detection[4])

            # Sadece belirli bir doğruluk eşiğinin üzerindeki tespitleri işle
            if confidence >= confidence_threshold:
                if label == 'YORGUN':
                    yorgunluk_frame_sayisi += 1

                    if yorgunluk_frame_sayisi >= yorgunluk_esik_degeri and not alarm_aktif:
                        # Alarm fonksiyonunu ayrı bir thread üzerinde çalıştır
                        alarm_thread_instance = threading.Thread(target=alarm_thread)
                        alarm_thread_instance.start()

                        # Yorgun durumu algılandı
                        yorgun_durumu_algildi = True

                        # Alarm aktif durumunu güncelle
                        alarm_aktif = True

                elif label == 'UYANIK':
                    uyanik_durumu_algildi = True

        # Eğer uyanık durumu algılandıysa yorgunluk frame sayısını sıfırla
        if uyanik_durumu_algildi:
            yorgunluk_frame_sayisi = 0
            alarm_aktif = False  # Uyanık durumu algılandığında veya eşik değeri aşıldığında alarmı kapat

        # Eğer alarm aktifse ve belirli bir süre geçtiyse "Dikkat! Yorgunsunuz!" yazısını göster
        if yorgunluk_frame_sayisi >= yorgunluk_esik_degeri:
            cv2.putText(frame, "Dikkat! Yorgunsunuz!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Her durumda frame sayısını göster
        message = f"Yorgunluk orani: {yorgunluk_frame_sayisi}"
        if yorgunluk_frame_sayisi > yorgunluk_esik_degeri:
            message = f"Yorgunluk orani: {yorgunluk_esik_degeri}"

        cv2.putText(frame, message, (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        if yorgunluk_frame_sayisi < yorgunluk_esik_degeri:
            # Çerçeve kalınlığını belirle, yorgunluk arttıkça kalınlığı artır
            frame_thickness = int(1 + 10 * np.exp(0.1 * yorgunluk_frame_sayisi))
            # Çerçeve rengi: (B, G, R), burada kırmızı renk
            frame_color = (0, 0, 255)
            # Çerçeve ekle
            cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), frame_color, frame_thickness)
        if yorgunluk_frame_sayisi > yorgunluk_esik_degeri:
            frame_thickness = int(1 + 10 * np.exp(0.1 * yorgunluk_esik_degeri))
            # Çerçeve rengi: (B, G, R), burada kırmızı renk
            frame_color = (0, 0, 255)
            # Çerçeve ekle
            cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), frame_color, frame_thickness)

    # 7. Sonuçları Göster
    cv2.imshow('YOLO', np.squeeze(results.render()))

    count += 1

    # 8. Çıkış İşlemi
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# 9. Temizleme
cap.release()
cv2.destroyAllWindows()

# deneme 6 threshold var 

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# düz tespit işlemi yapıyor. 